# 1. Data Collection - PART 2
## Notebook to scrap each page 
We generate links based on the stockno that we grabed and stored into car_list.csv file. to get further details we click the 'see all specifications' button and scrap some more 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import math
import os
chromedriver = f"{os.environ['HOME']}/.local/bin/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

### 1. Opening our car_list.csv

In [10]:
car_list=pd.read_csv('../data/car_list.csv')
car_list.drop('Unnamed: 0',axis =1, inplace = True)
print(car_list.shape)
car_list.tail()
                     

(1300, 5)


,stock_n,year,brand,model,price
1295,16328306,2013,Mini,Cooper,12599.0
1296,15238975,2015,Mini,Cooper Hardtop,14599.0
1297,16328154,2012,Mitsubishi,Eclipse GS Sport,11998.0
1298,16327884,2009,Mitsubishi,Lancer GTS,10599.0
1299,16328176,2012,Nissan,Altima S,11599.0


### 2. Setting up all the headings/parameters I need and an empty dataframe 

In [3]:
# list out all heading, parameters we will get from scraping a page. 
# add stock_no so that later on i can join the car_list.csv and car_details.csv  
heading =['stock_n','mileage','interior_color','exterior_color',\
          'drive_type','engine_size','num_cylinders','hp',\
          'torque','comp_ratio','cam_shaft','engine_type','bore',\
          'stroke','values_per_cylinder','fuel_cap','epa_mpg','wheel_base',\
          'length','width','height','curb_weight','leg_room',\
          'head_room','seating_cap','cargo_cap','tow_cap','pay_cap',\
          'gross','tire','trans']

#initialize an empty dataframe
df_cars=pd.DataFrame(columns=heading)
df_cars

,stock_n,mileage,interior_color,exterior_color,drive_type,engine_size,num_cylinders,hp,torque,comp_ratio,...,curb_weight,leg_room,head_room,seating_cap,cargo_cap,tow_cap,pay_cap,gross,tire,trans


### 2. Scrap it

In [4]:
#counter to check where we are, track if we ge interrupted
counter=1000

In [5]:
# condition for x_path
row_selector = '//div[@class ="card--text-block-list--row"]'


#loop through the car_list we have made.  maually set start and finish. Use exit counter to know where you are at 
for stock_num in car_list.loc[1000:1036,'stock_n']:
    
    # make link   
    link_str = "https://www.carmax.com/car/"+ str(stock_num)
        
    # open driver and link
    # note i open up the driver for every link i that use a get on because otherwise the site blocks me out. As long I
    # keep useing a new webdriver instance it seem to not kick me out !!! 
    driver = webdriver.Chrome(chromedriver)
    driver.get(link_str)
    time.sleep(3)
        
    # get car color interior and exterior
    int_color_selector = '//td[contains(text(),"Interior")]/following-sibling::td'
    ext_color_selector = '//td[contains(text(),"Exterior")]/following-sibling::td'
    
    ext_color = driver.find_element_by_xpath(ext_color_selector).text
    #print(ext_color)
    int_color = driver.find_element_by_xpath(int_color_selector).text
    #print(int_color)

    
    # grab mileage for a car
    miles_selector = '//div[@class ="price-mileage--mileage-container"]'
    temp_mile=[]
    for row in driver.find_elements_by_xpath(miles_selector):
        temp_mile.append(row.text)

    mileage = str(temp_mile[0]).split('\n')[1]
    #print(mileage)
    
    try:
        
        # get to full specs page
        go_button = driver.find_element_by_xpath('//a[contains(text(), "SEE ALL SPECIFICATIONS")]')
        go_button.click()
        
        print("clicking ....",counter)
        time.sleep(3);
        counter+=1
    
        # set up list with stuff already scrapped
        temp_list = []
        temp_list.append(stock_num)
        temp_list.append(mileage)
        temp_list.append(int_color)
        temp_list.append(ext_color)
    
        # start the heavy grabbing from the specs table
        for row in driver.find_elements_by_xpath(row_selector):
            #print (row.text)
            temp_str=row.text.rsplit('\n')[1]
            temp_list.append(temp_str)
        #print ("--",temp_list)
    
        #close window 
        driver.close()
    except:
        print('OOPS A .. no click')
        
    try:
        # put into a panda
        df_cars = df_cars.append(pd.DataFrame([temp_list], columns=heading),ignore_index=True)
    except:
        print('OPPS B ... no put in pandas')
    
print('Yeh!!!')
  

clicking .... 1000
clicking .... 1001
clicking .... 1002
clicking .... 1003
clicking .... 1004
clicking .... 1005
clicking .... 1006
clicking .... 1007
clicking .... 1008
clicking .... 1009
clicking .... 1010
clicking .... 1011
clicking .... 1012
clicking .... 1013
clicking .... 1014
clicking .... 1015
clicking .... 1016
clicking .... 1017
clicking .... 1018
clicking .... 1019
clicking .... 1020
clicking .... 1021
clicking .... 1022
clicking .... 1023
clicking .... 1024
clicking .... 1025
clicking .... 1026
clicking .... 1027
clicking .... 1028
clicking .... 1029
clicking .... 1030
clicking .... 1031
clicking .... 1032
clicking .... 1033
clicking .... 1034
clicking .... 1035
clicking .... 1036
Yeh!!!


In [6]:
# what have I got ?
print(counter)
print(df_cars.shape)
df_cars.tail()

1037
(37, 31)


,stock_n,mileage,interior_color,exterior_color,drive_type,engine_size,num_cylinders,hp,torque,comp_ratio,...,curb_weight,leg_room,head_room,seating_cap,cargo_cap,tow_cap,pay_cap,gross,tire,trans
32,16451277,36K,Black,Gray,2WDWD,2.0L engine,4 cylinders,188 horsepower@6000rpm,177 torque@4000rpm,12:1,...,3615 lbs.,"44.3""","39.2""",5/5,12 cu. ft.,N/A,N/A,N/R,N/A,N/A
33,16497371,31K,Black,Silver,2WDWD,2.3L engine,4 cylinders,310 horsepower@5500rpm,N/A,9.50,...,3532 lbs.,44.5'',37.6'',4/4,13.5 cu. ft.,N/A,N/A,N/A,N/A,N/A
34,16497374,29K,Black,Red,2WDWD,5.0L engine,8 cylinders,435 horsepower@6500rpm,N/A,11.00,...,3705 lbs.,44.5'',37.6'',4/4,13.5 cu. ft.,N/A,N/A,N/A,N/A,N/A
35,16450871,51K,Gray,White,2WDWD,3.7L engine,6 cylinders,270 horsepower@6500rpm,250 torque@4000rpm,11:1,...,6124 lbs.,N/R,N/R,12/15,107 cu. ft.,N/A,2810 lbs.,9000 lbs.,N/A,N/A
36,16450863,23K,Black,Brown,4WDWD,5.3L engine,8 cylinders,355 horsepower@5600rpm,383 torque@4100rpm,12:1,...,5414 lbs.,"45.3""","42.8""",5/5,N/R,2000/9100 lbs.,1876 lbs.,7200 lbs.,N/A,N/A


### 3. Write my data frame to file
If the file does not exist then i write the dataframe with headers else if the file is already there then don't append with the header row 

In [11]:
file_name = '../data/car_scrap.csv'
from pathlib import Path
config = Path('../data/car_scrap.csv')
if config.is_file():
    print('exist = yes')
    with open(file_name, 'a') as f:
        df_cars.to_csv(f, header=False)
    # Store configuration file values
else:
    # Keep presets
    print('exists = no')
    with open(file_name, 'a') as f:
        df_cars.to_csv(f, header=True)
    

exist = yes


### 4. Check it
read the file and check if stuff was written 

In [13]:
df_check = pd.read_csv('../data/car_scrap.csv')
df_check.shape

(1297, 32)

In [14]:
df_check.tail()

,Unnamed: 0,stock_n,mileage,interior_color,exterior_color,drive_type,engine_size,num_cylinders,hp,torque,...,curb_weight,leg_room,head_room,seating_cap,cargo_cap,tow_cap,pay_cap,gross,tire,trans
1292,22,16327849,51K,Gray,Silver,2WDWD,1.8L engine,4 cylinders,132 horsepower@6000rpm,128 torque@4400rpm,...,2855 lbs.,"42.3""","38.3""",5/5,13 cu. ft.,NaN,NaN,N/R,NaN,NaN
1293,23,16439674,36K,Gray,Silver,2WDWD,1.8L engine,4 cylinders,132 horsepower@6000rpm,128 torque@4400rpm,...,2855 lbs.,"42.3""","38.3""",5/5,13 cu. ft.,NaN,NaN,N/R,NaN,NaN
1294,24,16327401,30K,Black,White,2WDWD,1.8L engine,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1295,25,16328156,70K,Tan,Blue,2WDWD,1.8L engine,4 cylinders,98 horsepower@5200rpm,105 torque@4000rpm,...,3042 lbs.,"42.5""","38.6""",5/5,22 cu. ft.,NaN,NaN,N/R,NaN,NaN
1296,26,16328119,47K,Gray,Black,2WDWD,1.8L engine,4 cylinders,98 horsepower@5200rpm,105 torque@4000rpm,...,3042 lbs.,"42.5""","38.3""",5/5,22 cu. ft.,NaN,NaN,N/R,NaN,6-Speed Manual


### next -> clean_data.ipynb 